# Santander Value Prediction Challenge

Status - *Initial submission, possible improvements to come following some experimentation.*

Link to [Kaggle competion](https://www.kaggle.com/c/santander-value-prediction-challenge) 

#### Includes :
1. Feature Engineering
    * Additional features created. 
2. Preprocessing
    * Remove features with zero varience.
    * Where features are (nearly) perfectly correlated, remove one.
    * Apply log transform to features where the normality is improved by doing so.
    * MinMax Scale
    * Remove least important features
3. Ensemble of Gradient Boosting Regressor and Random Forest.

##   Import Required Packages

In [1]:
#General
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import pandas as pd


#Scikit-Learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


#Stats
from scipy.stats import linregress, shapiro

#Random Seed (for reproducibility)
seed = 42
np.random.seed = seed

##   Import Required Data

In [2]:
# Import Training Data
train = pd.read_csv('datasets/train.csv')
X = train.drop(['target', 'ID'], axis = 1)
y = round(np.log1p(train.target),32) #rounding off unnecessary precision.
ID = train.ID

In [3]:
# Import Competition Test Data
comp = pd.read_csv('datasets/test.csv')
X_comp = comp.drop(['ID'], axis = 1) 
ID_comp = comp.ID

## Performance Metric
Performance metric defined in the Kaggle competition.

In [4]:
def rmsle(y, pred):
    """Performance metric used in kaggle competition"""
    return np.sqrt(np.mean(np.power(np.log1p(y) - np.log1p(pred), 2)))


## Preprocessing Data

###   Feature Engineering 
Since all feature names are hashed it is not possible to test logical combinations for new features.   
However, all features seem to be monetary values. Therefore, various metrics of all these will be created.   
This will not harm prediction as features with minimal importance are removed later.

For `feature sum` and `sum_pos_neg_correlations` the direction of correlation for each feature is taken into account for the calculations. This is to cover the fact that some feature values could have a negative impact on the `target` i.e. debts.

In [5]:
class Additional_Features(TransformerMixin):
    """Additional predictor features created.
    
    feature_sum: Sum of all features factored by -1 or 1 
                 depending on that features correlation with the target
    median: Median of all fetures.
    max: Max of all features.
    var: Varience of values across all features.
    N_Not_0: Number of non zero features."""
    
    def __init__(self, **params):
        self.slope_dir = pd.DataFrame()
        self.cols = []
    
    def fit(self, X, y, **fit_params):
        print("Fitting Additional_Features")
        #For each feature the +ve or -ve correlation with the target is calculated
        self.cols = X.select_dtypes(include = 'number').columns
        self.slope_dir = pd.DataFrame(index=self.cols,
                          data = {'slope_dir': 0})                

        
        for col in self.cols:
            rows = X[col] != 0
            valid_rows = X[rows]
            
            # Unlikely to get reliable correlation from < 5 data points.
            if len(valid_rows) < 5 :
                continue
                
            # Check direction of correlation with target.
            # Target data already log_1p transformed, assumption is that log_1p(x) will give better linear fit.
            log_x = np.log1p(valid_rows.loc[:,col])
            LR = linregress(log_x, y[rows])
            if np.logical_or(np.isnan(LR.slope), np.isnan(LR.pvalue)):
                continue
            else:
                # Where p value of slope is < 0.1 correlation is taken as 0.
                self.slope_dir.loc[col,'slope_dir'] = (LR.slope * (LR.pvalue < 0.1)) / abs(LR.slope)
        return self
    
    def transform(self, X, **trans_params):
        print("Transforming Additional_Features")
        X_out = X.copy()
        #Sum of all features factored by their +/- correlation with target
        X_out['feature_sum'] = X.loc[:,self.cols].apply(lambda x: (x * self.slope_dir.loc[self.cols, 'slope_dir']).sum(), axis = 1)
        X_out['sum_pos_neg_correlations'] =  X.loc[:,self.cols].apply(lambda x: ((x != 0) * self.slope_dir.loc[self.cols, 'slope_dir']).sum(), axis = 1)
        #Further Fetures
        X_out['mean'] = X[self.cols].apply(lambda x: np.mean(x[x != 0]), axis = 1)        
        X_out['median'] = X[self.cols].apply(lambda x: np.median(x[x != 0]), axis = 1)
        X_out['max'] = X[self.cols].apply(lambda x: np.max(x[x != 0]), axis = 1)
        X_out['min'] = X[self.cols].apply(lambda x: np.min(x[x != 0]), axis = 1)        
        X_out['var'] = X[self.cols].apply(lambda x: np.var(x[x != 0]), axis = 1)
        X_out['N_Not_0'] = X[self.cols].apply(lambda x: (x != 0).sum(), axis = 1)
        
        
        # Remove NaN
        X_out[np.isnan(X_out)] = 0
                
        return X_out  

### Preprocessing Functions
To maintain a clean workflow (making use of a pipeline) all preprocessing is completed within custom transformer classes.  
All preprocessing is done within a single pipeline, outputing a pandas dataframe. Predictive modelling is completed separate to this, to remove the need to repeat the preprocessing while experimenting. 

Note that although `MinMaxScaler` is already compatible with pipelines, a custom version is created here to output a pandas dataframe rather than a numpy array.

In [6]:
#---------------------------Remove_Zero_Var---------------------------------------------#  
class Remove_Zero_Var(TransformerMixin):
    """Removes features with single values i.e. zero varience"""
    
    def __init__(self, **params):
        self.cols_with_0_var = []
        pass
            
    def fit(self, X, y, **fit_params):
        print("Fitting Remove_Zero_Var")
        self.cols_with_0_var = X.columns[X.nunique() == 1]
        return self
    
    def transform(self, X, **trans_params):
        print("Transforming Remove_Zero_Var")
        X_out = X.drop(self.cols_with_0_var, axis=1)
        return X_out
    
#--------------------------Remove_Correlated----------------------------------------------#
# Features with perfect correlation are hold redundant information.
# currently set to remove features where another has a correlation coefficient of >=0.99.
class Remove_Correlated(TransformerMixin):
    """Removes columns where another column is perfectly (>0.99) correlated."""
    def __init__(self, **params):
        self.redun_cols = []
        self.correlated_data = []
        self.num_cols = []
        pass
           
        
    def fit(self, X, y, **fit_params):
        print("Fitting Remove_Correlated")
        self.num_cols = X.select_dtypes(include = 'number').columns
        
        for i, col in enumerate(self.num_cols):
           
            rows = (X[col]!=0)
            
            # Unlikely to get reliable correlation from < 5 data points.
            # Features with < 5 values will likely be removed later by Remove_Least_Important.
            if sum(rows) < 5:
                continue
            
            # Feature Data
            temp_1 = X.loc[rows, col]
            
            # Cycle over all other features to check for correlation
            for j in np.arange(i + 1, len(self.num_cols)):
                temp_2 = X.loc[rows, self.num_cols[j]]
                
                # No 0's in temp_1 so correlation is unlikely.
                if (temp_2 == 0).any():
                    continue
                # Check correlation
                if np.corrcoef(temp_1, temp_2)[0][1] >= 0.99:
                    self.redun_cols.append(col)
                    self.correlated_data.append((temp_1,temp_2))
                    break
                else:
                    pass
            
        return self
    
    
    def transform(self, X, **trans_params):
        print("Transforming Remove_Correlated")
        X_out = X.drop(self.redun_cols, axis=1)
        return X_out
    
#----------------------------Remove_Least_Important--------------------------------------------# 
# Remove features with an importance rating less than the threshold passed.
# Based on a random forest regressor.
# Threshold set relativly low (10^-5) to ensure minimal loss of useful information.
class Remove_Least_Important(TransformerMixin):
    """Finds and removes least important features.
    
    Based on a Random Forest Regressor.
    threshold: features with an importance rating below this are removed."""
    
    def __init__(self, threshold=0.0001, random_seed=42, **params):
        self.threshold=threshold
        self.unimportant_cols = []
        self.importance = pd.DataFrame()
        self.random_seed = random_seed
            
    def fit(self, X, y, **fit_params):
        print("Fitting Remove_least_important")
        model = RandomForestRegressor(random_state=self.random_seed)
        model.fit(X, y)
        
        self.importance = pd.DataFrame({'col':X.columns,
                                   'importance':model.feature_importances_})\
                                    .sort_values(by='importance', ascending=False)

        self.unimportant_cols = self.importance.loc[self.importance['importance'] < self.threshold, 'col'].values
        return self
    
    
    def transform(self, X, **trans_params):
            print("Transforming Remove_least_important")
            X_out = X.drop(self.unimportant_cols, axis=1)
            return X_out

#-------------------------Log_Trans_X-----------------------------------------------#          
# For each feature the shapiro normality metric is calculated.
# A log tranform is applied if this metric is improved by it, else it is unchanged.
# Currently resets any inf values to 0 - to be reviewed.

class Log_Trans_X(TransformerMixin):
    """Apply np.log1p to feature where it's shapiro normality metric improves by doing so."""
    
    def __init__(self, **params):
        from scipy.stats import shapiro
        self.col_to_be_log = []
        self.num_cols = []
            
    def fit(self, X, y, **fit_params):
        print("Fitting Log_Trans_X")
        self.num_cols = X.select_dtypes(include = 'number').columns
        for col in self.num_cols:
            temp_orig = X.loc[X[col]!=0, col]
            
            # Ensure all positive values
            if temp_orig.min() < 0:
                temp_orig = temp_orig - temp_orig.min()
            # Ignore features with minimal data   
            if np.logical_or(len(temp_orig)<=3, temp_orig.nunique()==1):
                continue            
            else:
                temp_log = np.log1p(temp_orig)
                
                temp_orig_shapiro = shapiro(temp_orig)[0]
                temp_log_shapiro = shapiro(temp_log)[0]
                
                if temp_log_shapiro > temp_orig_shapiro:
                    self.col_to_be_log.append(col)                
                
        return self
    
    def transform(self, X, **trans_params):
            print("Transforming Log_Trans_X")
            X_out = X.copy()
            for col in self.col_to_be_log:
                # Ensure all positive values
                if X_out[col].min() < 0:
                    X_out[col] = X_out[col] - X_out[col].min() 
                # Log values
                X_out[col] = np.log1p(X_out[col])
            # Remove errors
            X_out[np.isinf(X_out)] = 0
            X_out[np.isnan(X_out)] = 0
            return X_out
   
             
#---------------------MinMax_Df---------------------------------------------------#
# Custom transform to output a pandas dataframe rather than a numpy array.
class MinMax_Df(TransformerMixin):
    """Custom transformer to apply MinMax scaling and return a dataframe."""
    
    def __init__(self, **params):
        from sklearn.preprocessing import MinMaxScaler
        self.scaler = MinMaxScaler()
            
    def fit(self, X, y, **fit_params):
        print("Fitting MinMax_Df")
        self.scaler.fit(X, y)
        return self
    
    def transform(self, X, **trans_params):
        print("Transforming MinMax_Df")
        data = self.scaler.transform(X)
        X_out = pd.DataFrame(index = X.index,
                            columns = X.columns,
                            data = data)
        return X_out

#------------------------Round------------------------------------------------# 
# Removal of unnecessary precision.
class Round(TransformerMixin):
    """Rounds all values off to npd decimal places. Further precision is unnecessary."""
    
    def __init__(self, ndp, **params):
        self.ndp = ndp
        pass

    def fit(self, X, y, **fit_params):
        print("Fitting Round")
        return self
    
    def transform(self, X, **trans_params):
        print("Transforming Round")
        X_out = X.round(self.ndp)
        return X_out

### Preprocessing Pipe
All feature engineering and preprocessing is organised into one pipeline.

In [7]:
preprocess_pipe = Pipeline([
        ('remove_zero_varience', Remove_Zero_Var()),
        ('added_features', Additional_Features()),    
        ('log_transform_X', Log_Trans_X()),    
        ('minmax_scale', MinMax_Df()),
        ('round', Round(32)),
        ('remove_least_important', Remove_Least_Important(threshold=10**-5, random_seed=seed)),
        ('remove_correlating_feats', Remove_Correlated())
        ])


In [8]:
X_clean = preprocess_pipe.fit_transform(X,y)

Fitting Remove_Zero_Var
Transforming Remove_Zero_Var
Fitting Additional_Features


C:\Users\Rich\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:107: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
C:\Users\Rich\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:119: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


Transforming Additional_Features
Fitting Log_Trans_X
Transforming Log_Trans_X
Fitting MinMax_Df
Transforming MinMax_Df
Fitting Round
Transforming Round
Fitting Remove_least_important
Transforming Remove_least_important
Fitting Remove_Correlated
Transforming Remove_Correlated


In [19]:
X_clean.to_csv('datasets/0__X_clean.csv')

## Modelling

### Cross Validation Evaluation
Function to apply kaggle performance metric on potential models.

In [20]:
def cv_rmsle(model, X, y, n_tests=5, seed=None):
    scores = []
    
    for i in np.arange(n_tests):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = seed + i)
        model.fit(X_train, y_train)
        pred = np.expm1(model.predict(X_test))
        scores.append(rmsle(np.expm1(y_test), pred))
    return scores

### Gradient Boosting Regressor
Sereval iterations of GridSearch completed on a limited number of hyperparameters.


In [21]:
# Gradient Boosting Model
#GBR = GradientBoostingRegressor()

#### Grid Search
# GBR_tuning_params = [{'n_estimators':[30, 50, 70],
#                  'max_depth':[6, 9, 12],
#                  'learning_rate':[0.03, 0.05, 0.07],
#                  'max_features' : [750, 1000, None]
#                  }]

# GBR_Search = GridSearchCV(GBR, 
#                       GBR_tuning_params,
#                       cv=3, 
#                       scoring= 'neg_mean_squared_error',
#                      verbose = 5,
#                          n_jobs = 8)
# GBR_Search.fit(X_clean, y)
#GBR_Search.best_params_
# Results learning rate = 0.05 , max_depth = 9 , n_estimators = 70, max_features = 1000 

In [22]:
## Tuned Model
GBR = GradientBoostingRegressor(n_estimators=70, 
                                learning_rate=0.05,
                                max_depth=9,
                                max_features=1000,
                               random_state = seed)


In [23]:
# Check tuned model with cross validation
error_GBR = cv_rmsle(GBR, X_clean, y, seed = seed)
print(np.mean(error_GBR))


1.3489227848059147


### Random Forest Regressor
Sereval iterations of GridSearch completed on a limited number of hyperparameters.

In [24]:
# RFR = RandomForestRegressor(n_jobs=-1)

# ### Grid Search
# RFR_tuning_params = [{'n_estimators':[300, 500, 800],
#                  'max_depth':[15, 20, 30, None],
#                  'max_features' : [500, 1000, 1500]
#                  }]

# RFR_Search = GridSearchCV(RFR, 
#                       RFR_tuning_params,
#                       cv=3, 
#                       scoring= 'neg_mean_squared_error',
#                       verbose = 3)
# RFR_Search.fit(X_clean, y)
#RFR_Search.best_params_
#Results max_features = 800 , max_depth = 15, n_estimaters = 500

In [25]:
## Tuned Model
RFR = RandomForestRegressor(max_features=800,
                           max_depth = 15,
                           n_estimators = 500,
                           random_state = seed)

In [26]:
# Check tuned model with cross validation
error_RFR = cv_rmsle(RFR, X_clean, y, seed = seed)
print(np.mean(error_RFR))

1.3370990857851945


### Ensemble

In [27]:
class Ensemble_Model(BaseEstimator):  
    """Ensemble of regression models"""

    def __init__(self, models, **Params):
        self.models = models
        

    def fit(self, X, y=None):
        for m in self.models:
            m.fit(X,y)
        return self

    def predict(self, X, y=None):
        y_pred = np.zeros(len(X))
        
        for m in self.models:
            indv_model = np.expm1(np.array(m.predict(X)))
            y_pred += indv_model
            
        y_pred = np.log1p((list(y_pred/len(self.models))))   
            
        return y_pred


In [28]:
Ensemble = Ensemble_Model(models=[GBR, RFR])
error_ensemble = (cv_rmsle(Ensemble, X_clean, y, seed = seed))
print(np.mean(error_ensemble))

1.3377839173717652


## Competition Submission

In [29]:
# Use pre-processing pipeline on competition data
X_comp_clean = preprocess_pipe.transform(X_comp)

Transforming Remove_Zero_Var
Transforming Additional_Features


C:\Users\Rich\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Rich\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Transforming Log_Trans_X
Transforming MinMax_Df
Transforming Round
Transforming Remove_least_important
Transforming Remove_Correlated


In [30]:
X_comp_clean.to_csv('datasets/0__X_comp_clean.csv')

In [31]:
# Train on all data
Ensemble_comp = Ensemble_Model(models = [RFR, GBR])
Ensemble_comp.fit(X_clean, y)
y_comp = np.expm1(Ensemble_comp.predict(X_comp_clean))


comp_sumbmission = pd.DataFrame({'ID' : ID_comp,
                                'target' : y_comp})
comp_sumbmission.to_csv('0__Submission.csv',
                       index = False)


## Competition Score - 1.40